In [1]:
import torchtext
from torchtext.vocab import Vectors, GloVe
import torch
import torch.autograd as autograd
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
from submission.models import *
from evaluation import *

In [2]:
# Our input $x$
TEXT = torchtext.data.Field()
    
# Our labels $y$
LABEL = torchtext.data.Field(sequential=False)


train, val, test = torchtext.datasets.SST.splits(
    TEXT, LABEL,
    filter_pred=lambda ex: ex.label != 'neutral')

# Build vocab
TEXT.build_vocab(train)
LABEL.build_vocab(train)

train_iter, val_iter, test_iter = torchtext.data.BucketIterator.splits(
(train, val, test), batch_size=11, device=-1)

# Build the vocabulary with word embeddings
url = 'https://s3-us-west-1.amazonaws.com/fasttext-vectors/wiki.simple.vec'
TEXT.vocab.load_vectors(vectors=Vectors('wiki.simple.vec', url=url))

In [3]:
test_iter = torchtext.data.BucketIterator(test, train=False, batch_size=10,
                                          device=-1)
model = LogisticRegression(TEXT, LABEL)
print(model_eval(model, test))
# trainer = TextTrainer(TEXT, LABEL, model)
# trainer.train(train_iter, num_iter=1000, skip_iter=100)

(540, 1821)


In [7]:
model = MultinomialNB(TEXT, LABEL)
model.train(train_iter)

0
100
200
300
400
500
600


In [5]:
model_save_predictions(model, test_iter)